In [24]:
from PIL import Image
import os
import re
from IPython.display import display
import json
import numpy as np
from tqdm import tqdm

In [26]:
class_annotations_file = "/root/repos/vlmaps/data/rugd/RUGD_annotations/RUGD_annotation-colormap.txt"
with open(class_annotations_file, "r") as f:
    class_annotations = f.readlines()

CLASS_COLORS = {}
for line in class_annotations:
    idx, class_name, *color = line.strip().split()
    CLASS_COLORS[class_name] = [int(c) for c in color]

COLOR_TO_CLASS_IDX = {tuple(CLASS_COLORS[k]): idx for idx, k in enumerate(CLASS_COLORS.keys())}

In [23]:
import rerun as rr
import time
rr.init("rugd_trail", recording_id="rugd_trail")
current_time = time.strftime("%Y%m%d_%H%M%S")
rr.save(f"/root/repos/vlmaps/data/rugd/rerun/rugd_{current_time}.rrd")

# Assign a label and color to each class
labels_mapping = [
    rr.AnnotationInfo(id=i, label=label, color=CLASS_COLORS[label])
    for i, label in enumerate(CLASS_COLORS.keys())
]
rr.log("rugd_trail", rr.AnnotationContext(labels_mapping), static=True)

In [30]:
# Pre-compute the entire lookup table once
def create_color_lookup_table():
    # Create a 256x256x256 lookup table
    lookup = np.zeros((256, 256, 256), dtype=np.int32)
    
    for class_name, color in CLASS_COLORS.items():
        class_idx = list(CLASS_COLORS.keys()).index(class_name)
        lookup[color[0], color[1], color[2]] = class_idx
    
    return lookup
color_lookup = create_color_lookup_table()

In [31]:
trail_datasets = ["trail"] + ["trail-" + str(i) for i in range(3, 8)] + ["trail-" + str(i) for i in range(9, 16)]
trail_datasets = ["trail-15"]

img_dataset_dir = "/root/repos/vlmaps/data/rugd/RUGD_frames-with-annotations"
anno_dataset_dir = "/root/repos/vlmaps/data/rugd/RUGD_annotations"

for trail_dataset in trail_datasets:
    print(f"####### Processing {trail_dataset}")
    img_dir = os.path.join(img_dataset_dir, trail_dataset)
    anno_dir = os.path.join(anno_dataset_dir, trail_dataset)

    img_files = [f for f in os.listdir(img_dir) if f.endswith('.png')]
    anno_files = [f for f in os.listdir(anno_dir) if f.endswith('.png')]

    for img_file, anno_file in tqdm(zip(img_files, anno_files), desc="Processing images and annotations"):
        pattern = f"{trail_dataset}_(\\d+)\\.png"
        match = re.match(pattern, img_file)
        if not match:
            print(f"Warning: Filename {img_file} does not match expected pattern")
            continue
        idx = match.group(1)
        rr.set_time(timeline="frame", sequence=int(idx))
        img_path = os.path.join(img_dir, img_file)
        anno_path = os.path.join(anno_dir, anno_file)

        img = Image.open(img_path)
        anno = Image.open(anno_path)

        anno_array = np.array(anno)

        seg_map = color_lookup[anno_array[:, :, 0], anno_array[:, :, 1], anno_array[:, :, 2]]

        rr.log(f"rugd_trail/{trail_dataset}/img", rr.Image(img))
        rr.log(f"rugd_trail/{trail_dataset}/anno", rr.SegmentationImage(seg_map))

####### Processing trail-15


Processing images and annotations: 324it [00:08, 38.51it/s]
